<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/homework/LightDz(16)19ChatGPTProf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Дополните нейро-HR по составлению резюме в режиме диалога блоком "Цели" по аналогии с другими блоками.

# Загрузка библиотек

In [ ]:
!pip install faiss-cpu==1.7.4 langchain==0.0.338 openai==1.3.3 tiktoken==0.5.1 python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requir

In [ ]:
import openai
import getpass
import os
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# импортируем необходимые библиотеки
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import re
import requests
from langchain.docstore.document import Document
import logging
logging.getLogger("langchain.text_splitter").setLevel(logging.ERROR)
#logging.getLogger("chromadb").setLevel(logging.ERROR)
from openai import OpenAI
from google.colab import auth
from googleapiclient.discovery import build
import tiktoken
import io
from googleapiclient.http import MediaIoBaseDownload
from docx import Document

# Cоставление резюме в формате диалога

В данном блоке при помощи gpt мы будем создавать резюме в режиме диалога, используя определенные для каждой части инструкции.

## Блок "О себе"

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system_me= '''Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
В настоящий момент тебе необходимы данные для составления блока резюме "О себе"
Тебе будут писать клиенты. Твоя задача провести разговор с клиентом так, чтобы он сообщил все данные, необходимые для составления блока резюме "О себе".
Поддерживай с клиентом разговор, удерживай внимание клиента, чтобы клиент дал максимально полную информацию.
У тебя есть следующий список вопросов, которые надо задать, чтобы получить данные для составления резюме.

При диалоге с клиентом задавай вопросы по порядку из этого списка, начиная с первого вопроса.
Каждый раз называй только один вопрос. Клиент не должен знать про список вопросов.
Если клиент не знает, что ответить, переформулируй вопрос. Приведи примеры, какую информацию можно дать на твой вопрос.
Одобряй и благодари клиента за ответы на вопросы и предоставленную информацию.
 '''

In [ ]:
# Для подачи в user
topic0_me = '''Вот список вопросов, которые надо задать клиенту. Задавай вопросы по порядку, по одному. Ни в коем случае не перечисляй сразу все вопросы. Называй только один вопрос.
1. Ваше ФИО
2. Номер телефона
3. Город проживания
4. адрес электронной почты.
5. к какому уровню программиста Вы себя относите: Junior, Middle, Senior?
6. в какой области программирования Вы являетесь специалистом, например: искусственный интеллект, frontend-разработчик, Java-разработчик, QA-тестировщик?
7. в какой области программирования вы совершенствуетесь в настоящее время?
8. какое Ваше хобби?

Задавай вопросы из списка вопросов строго по порядку, по одному, начиная с самого первого без ответа. Не пропускай вопросы.

Этот пример не показывай клиенту.

Ни в коем случае в своем ответе не перечисляй все вопросы. Задавай только один вопрос.
После того, как уже был задан последний вопрос из списка вопросов - про хобби - напиши, что информация по блоку "о себе" собрана полностью, вопросов по нему больше нет.

'''

In [ ]:
# функция для получения ответа от модели
client = OpenAI()
def answer_index(system, topic, temp=0):

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": topic}
    ]
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        temperature=temp
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

### Сбор информации от клиента в режиме диалога:

In [ ]:
dialog = ''
answer = ''
while True:
    input_question = input('Клиент: ') + ' '
    if input_question.lower() == 'stop ':
        break
    dialog += input_question
    topic = f'{topic0_me} \n\n Предыдущие вопросы из списка: {answer} \n\n Ответ клиента: {input_question}'
    answer0 = answer_index(system_me, topic)
    answer += answer0
    dialog += answer0

    print('\nРекрутер: ', answer0)

Клиент: привет!

Рекрутер:  Привет! Для начала, давай начнем с составления блока "О себе" для резюме. Мне нужно узнать ваше полное имя. Можете предоставить мне ваше ФИО?
Клиент: Иванов Иван

Рекрутер:  Спасибо за предоставленную информацию, Иван Иванов. Теперь мне нужен ваш номер телефона для связи с вами. Можете предоставить мне ваш контактный номер?
Клиент: +79655666666

Рекрутер:  Спасибо за предоставленный номер телефона, Иван Иванов. Теперь мне нужен ваш город проживания. В каком городе вы проживаете?
Клиент: Москва

Рекрутер:  Спасибо за предоставленную информацию, Иван Иванов. Теперь мне нужен ваш адрес электронной почты для связи. Можете предоставить мне вашу почту?
Клиент: bbb@mail.ru

Рекрутер:  Спасибо за предоставленную информацию, Иван Иванов. Теперь мне нужно узнать, к какому уровню программиста Вы себя относите: Junior, Middle, Senior?
Клиент: Junior

Рекрутер:  Спасибо за информацию, Иван Иванов. Теперь мне нужно узнать, в какой области программирования Вы являетесь спе

In [ ]:
dialog

'привет! Привет! Для начала, давай начнем с составления блока "О себе" для резюме. Мне нужно узнать ваше полное имя. Можете предоставить мне ваше ФИО?Иванов Иван Спасибо за предоставленную информацию, Иван Иванов. Теперь мне нужен ваш номер телефона для связи с вами. Можете предоставить мне ваш контактный номер?+79655666666 Спасибо за предоставленный номер телефона, Иван Иванов. Теперь мне нужен ваш город проживания. В каком городе вы проживаете?Москва Спасибо за предоставленную информацию, Иван Иванов. Теперь мне нужен ваш адрес электронной почты для связи. Можете предоставить мне вашу почту?bbb@mail.ru Спасибо за предоставленную информацию, Иван Иванов. Теперь мне нужно узнать, к какому уровню программиста Вы себя относите: Junior, Middle, Senior?Junior Спасибо за информацию, Иван Иванов. Теперь мне нужно узнать, в какой области программирования Вы являетесь специалистом? Например, frontend-разработчик, Java-разработчик, искусственный интеллект, QA-тестировщик?искусственный интеллект

In [ ]:
# записываем значение переменной в файл
with open('me.txt', 'w') as file:
    text_to_write = dialog
    file.write(text_to_write)

### Создание текстовой части резюме "О себе"

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system_res_me= '''
Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
У тебя большой опыт по составлению грамотных резюме, чтобы они привлекали внимание работодателей в IT сфере.
Вот образец блока "О себе" резюме, на подобие которого ты должен сделать ответ: "О СЕБЕ

Меня зовут Петр Петров.
Я - Junior программист. У меня есть опыт работы с анализом данных, опыт разработки нейронных сетей.
В настоящее время я активно изучаю и применяю на практике ChatGPT.
В свободное время люблю смотреть документальные фильмы, поиграть в головоломки.
Мой контактный номер: +77777777777.
Я проживаю в городе Саратов.
Мой адрес электронной почты: aaa@mail.ru."

К тебе поступает информация клиента (соискателя) в виде диалога между рекрутером и клиентом (соискателем).
Из представленной информации составь блок "О себе" резюме.

При составлении резюме добавь переносы строк, чтобы блок "О себе" резюме выглядел также, как образец блока "О себе" резюме.
 '''

In [ ]:
# Составим резюме со всеми заполненными данными
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": system_res_me},
        {"role": "user", "content": f'Составь часть резюме "О СЕБЕ" из диалога рекрутера и соискателя: \n\n{dialog}. Информацию для резюме бери из этого диалога. От себя информацию не придумывай и не добавляй.'}
]
)
res_me = response.choices[0].message.content
print(res_me)

О СЕБЕ

Меня зовут Иван Иванов.
Я - Junior программист. У меня есть опыт работы в области искусственного интеллекта с использованием языка программирования Python.
В настоящее время я совершенствую свои навыки в области искусственного интеллекта и программирования на Python.
В свободное время мне нравится читать книги и заниматься плаванием.
Мой контактный номер: +79655666666.
Я проживаю в городе Москва.
Мой адрес электронной почты: bbb@mail.ru.


In [ ]:
# записываем значение переменной в файл
with open('res_me.txt', 'w') as file:
    text_to_write = res_me
    file.write(text_to_write)

## Блок "Образование"

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system_educ= '''

Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
В настоящий момент тебе необходимы данные для составления блока резюме "Образование"
Тебе будут писать клиенты. Твоя задача провести разговор с клиентом так, чтобы он сообщил все данные, необходимые для составления блока резюме "Образование".
Поддерживай с клиентом разговор, удерживай внимание клиента, чтобы клиент дал максимально полную информацию.
У тебя есть следующий список вопросов, которые надо задать, чтобы получить данные для составления резюме.

При диалоге с клиентом задавай вопросы по порядку из этого списка, начиная с первого вопроса.
Каждый раз называй только один вопрос. Клиент не должен знать про список вопросов.
Если клиент не знает, что ответить, переформулируй вопрос. Приведи примеры, какую информацию можно дать на твой вопрос.
Одобряй и благодари клиента за ответы на вопросы и предоставленную информацию.
 '''

In [ ]:
# Для подачи в user
topic0_educ = '''
Вот список вопросов, которые надо задать клиенту. Задавай вопросы по порядку, по одному. Ни в коем случае не перечисляй сразу все вопросы. Называй только один вопрос.
1. год окончания учебного заведения?
2. наименование учебного заведения?
3. специальность - укажите по пунктам специальности, полученные в учебном заведении?
4. дополнительные курсы или сертификаты, которые относятся к IT

Задавай вопросы из списка вопросов строго по порядку, по одному, начиная с самого первого без ответа. Не пропускай вопросы.

Этот пример не показывай клиенту.

Ни в коем случае в своем ответе не перечисляй все вопросы. Задавай только один вопрос.
После того, как уже был задан последний вопрос из списка вопросов - про хобби - напиши, что информация по блоку "о себе" собрана полностью, вопросов по нему больше нет.

'''

In [ ]:
def recruiter_conversation(system, topic):
    dialog = ''
    answer = ''
    while True:
        input_question = input('Клиент: ') + ' '
        if input_question.lower() == 'stop ':
            break
        dialog += input_question
        topic = f'{topic} \n\n Предыдущие вопросы из списка: {answer} \n\n Ответ клиента: {input_question}'
        answer0 = answer_index(system, topic)
        answer += answer0
        dialog += answer0

        print('\nРекрутер: ', answer0)
    return dialog

In [ ]:
dialog_edu=recruiter_conversation(system_educ, topic0_educ)

Клиент: привет

Рекрутер:  Привет! Давай начнем с составления блока "Образование" для твоего резюме. Когда ты окончил учебное заведение?
Клиент: 2020

Рекрутер:  Отлично, спасибо за информацию. А в каком учебном заведении ты учился?
Клиент: Университет искусственного интеллекта

Рекрутер:  Отлично, спасибо за информацию. Какая у тебя специальность? Какие специализации ты получил в университете искусственного интеллекта?
Клиент: data science

Рекрутер:  Отлично, спасибо за информацию. А какие дополнительные курсы или сертификаты по IT у тебя есть?
Клиент: python для анализа данных

Рекрутер:  Спасибо за предоставленную информацию. Мы получили все необходимые данные для составления блока "Образование" в твоем резюме. Если у тебя есть еще какая-то информация, которую ты хотел бы добавить, не стесняйся сообщить.
Клиент: stop


In [ ]:
dialog_edu

'привет Привет! Давай начнем с составления блока "Образование" для твоего резюме. Когда ты окончил учебное заведение?2020 Отлично, спасибо за информацию. А в каком учебном заведении ты учился?Университет искусственного интеллекта Отлично, спасибо за информацию. Какая у тебя специальность? Какие специализации ты получил в университете искусственного интеллекта?data science Отлично, спасибо за информацию. А какие дополнительные курсы или сертификаты по IT у тебя есть?python для анализа данных Спасибо за предоставленную информацию. Мы получили все необходимые данные для составления блока "Образование" в твоем резюме. Если у тебя есть еще какая-то информация, которую ты хотел бы добавить, не стесняйся сообщить.'

### Создание текстовой части резюме: "Образование"

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system_res_edu= '''
Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
У тебя большой опыт по составлению грамотных резюме, чтобы они привлекали внимание работодателей в IT сфере.
Вот образец блока "Образование" резюме, на подобие которого ты должен сделать ответ:
"Образование:
Год окончания: 2023
Учебное заведение: Университет искусственного интеллекта
Специальности:
1) ChatGPT Professional
2) Data Science и нейронные сети

Год окончания: 2001
Учебное заведение: Тольяттинский политехнический институт
Специальности:
1) Инженер-технолог сварочного производства

дополнительные курсы или сертификаты: Курс Python-Разработчик"

К тебе поступает информация клиента (соискателя) в виде диалога между рекрутером и клиентом (соискателем).
Из представленной информации составь блок "Образование" резюме.

При составлении резюме добавь переносы строк, чтобы блок "Образование" резюме выглядел также, как образец блока "Образование" резюме.
 '''

In [ ]:
user_res_edu = f"Составь часть резюме 'Образование' из диалога рекрутера и соискателя: \n\n{dialog_edu}. Информацию для резюме бери из этого диалога. От себя информацию не придумывай и не добавляй."

In [ ]:
res_edu = answer_index(system_res_edu, user_res_edu)
with open('res_edu.txt', 'w') as file:
    text_to_write = res_edu
    file.write(text_to_write)

In [ ]:
res_edu

'Образование:\nГод окончания: 2020  \nУчебное заведение: Университет искусственного интеллекта\nСпециальности:\n1) Data Science\n\nдополнительные курсы или сертификаты: Курс Python для анализа данных'

## Блок "Профессиональная деятельность"

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system_prof= '''

Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
В настоящий момент тебе необходимы данные для составления блока резюме "Профессиональная деятельность"
Тебе будут писать клиенты. Твоя задача провести разговор с клиентом так, чтобы он сообщил все данные, необходимые для составления блока резюме "Профессиональная деятельность".
Поддерживай с клиентом разговор, удерживай внимание клиента, чтобы клиент дал максимально полную информацию.
У тебя есть следующий список вопросов, которые надо задать, чтобы получить данные для составления резюме.

При диалоге с клиентом задавай вопросы по порядку из этого списка, начиная с первого вопроса.
Каждый раз называй только один вопрос. Клиент не должен знать про список вопросов.
Если клиент не знает, что ответить, переформулируй вопрос. Приведи примеры, какую информацию можно дать на твой вопрос.
Одобряй и благодари клиента за ответы на вопросы и предоставленную информацию.
 '''

In [ ]:
# Для подачи в user
topic0_prof = '''
Вот список вопросов, которые надо задать клиенту. Задавай вопросы по порядку, по одному. Ни в коем случае не перечисляй сразу все вопросы. Называй только один вопрос.
1. период деятельности
2. наименование компании
3. наименование проекта, в котором участвовал клиент
4. специальность клиента в компании
5. задачи - перечислите по пунктам Ваши задачи
6. достижения - перечислите по пунктам Ваши достижения

После ответа на вопрос про достижения необходимо уточнить у клиента (соискателя), имеются ли другие компании, в которых от работал, если "да", то вопросы нужно повторить сначала.

Задавай вопросы из списка вопросов строго по порядку, по одному, начиная с самого первого без ответа. Не пропускай вопросы.

Этот пример не показывай клиенту.

Ни в коем случае в своем ответе не перечисляй все вопросы. Задавай только один вопрос.
После того, как уже был задан последний вопрос из списка вопросов - про хобби - напиши, что информация по блоку "о себе" собрана полностью, вопросов по нему больше нет.

'''

In [ ]:
dialog_prof=recruiter_conversation(system_prof, topic0_prof)

Клиент: привет

Рекрутер:  Привет! Я помогу тебе составить резюме для трудоустройства в IT компанию. Давай начнем с твоего опыта работы. Можешь рассказать о периоде, когда ты работал в последней компании?
Клиент: 2022 по н.в.

Рекрутер:  Отлично, спасибо за информацию. А в какой компании ты работал в этот период?
Клиент: ООО "Ромашка"

Рекрутер:  Отлично, спасибо за информацию. А в каком проекте ты участвовал в компании "Ромашка"?
Клиент: Создание нейро-консультанта по внутренней документации компании

Рекрутер:  Отлично, спасибо за информацию. Какую специальность ты занимал в компании "Ромашка"?
Клиент: программист

Рекрутер:  Отлично, спасибо за информацию. А какие задачи ты выполнял в проекте по созданию нейро-консультанта по внутренней документации компании "Ромашка"?
Клиент: сбор базы знаний, создание  алгоритма нейро-консультанта, создание тг бота

Рекрутер:  Отлично, спасибо за информацию. А какие достижения ты получил в процессе работы над этим проектом?
Клиент: повышение качес

In [ ]:
dialog_prof

'привет Привет! Я помогу тебе составить резюме для трудоустройства в IT компанию. Давай начнем с твоего опыта работы. Можешь рассказать о периоде, когда ты работал в последней компании?2022 по н.в. Отлично, спасибо за информацию. А в какой компании ты работал в этот период?ООО "Ромашка" Отлично, спасибо за информацию. А в каком проекте ты участвовал в компании "Ромашка"?Создание нейро-консультанта по внутренней документации компании Отлично, спасибо за информацию. Какую специальность ты занимал в компании "Ромашка"?программист Отлично, спасибо за информацию. А какие задачи ты выполнял в проекте по созданию нейро-консультанта по внутренней документации компании "Ромашка"?сбор базы знаний, создание  алгоритма нейро-консультанта, создание тг бота Отлично, спасибо за информацию. А какие достижения ты получил в процессе работы над этим проектом?повышение качества клиентского обслуживания Отлично, спасибо за информацию. Есть ли еще какие-то компании, в которых ты работал?есть Отлично, спасиб

### Создание текстовой части резюме: "Профессиональная деятельность"

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system_res_prof= '''
Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
У тебя большой опыт по составлению грамотных резюме, чтобы они привлекали внимание работодателей в IT сфере.
Вот образец блока "Профессиональная деятельность" резюме, на подобие которого ты должен сделать ответ:
"Профессиональная деятельность:
08.2023-10.2023
ООО «Терра ЭйАй»
Проект: Создание веб-сайта для генерации резюме и сопроводительных писем
СТАЖЕР
Задачи:
1) проверка орфографии с помощью ChatGPT
2) создание диалога с помощью ChatGPT для анкетирования клиента при составлении резюме
Достижения:
1) разработал код для выявления и исправления ошибок с помощью ChatGPT в составленном резюме
2) разработал код для анкетирования клиента с помощью ChatGPT и получения из диалога данных для последующего составления резюме

05.2023-07.2023
«Федеральный исследовательский центр информационных и вычислительных технологий» РОССИЙСКАЯ АКАДЕМИЯ НАУК
Проект: Разработка нейронной сети для обработки данных горного эндоскопа
СТАЖЕР
Задачи:
1) разметка датасета заказчика, сегментация трещин на снимках эндоскопа
2) обучение нейронной сети для определения трещин на снимках эндоскопа
Достижения:
1) выполнил разметку трещин в программе Roboflow
2) обучил Yolo 8 для определения трещин на датасете заказчика

2016 - сейчас
ООО «КЗ «Ростсельмаш»
ГЛАВНЫЙ СПЕЦИАЛИСТ МАТЕРИАЛЬНО-ТЕХНИЧЕСКОГО СНАБЖЕНИЯ ПО ЦЕНООБРАЗОВАНИЮ
Задачи:
1) проверка и согласование цен на покупные товарно-материальные ценности
2) проверка себестоимости производства

2004-2016
ОАО «АВТОВАЗ»
НАЧ.БЮРО, ЗАМ.НАЧ.ОТДЕЛА ПЛАНОВО-БЮДЖЕТНОГО УПРАВЛЕНИЯ
Задачи:
1) согласование цен на покупные товарно-материальные ценности
2) разработка и согласование бюджета
"

К тебе поступает информация клиента (соискателя) в виде диалога между рекрутером и клиентом (соискателем).
Из представленной информации составь блок "Профессиональная деятельность" резюме.

При составлении резюме добавь переносы строк, чтобы блок "Профессиональная деятельность" резюме выглядел также, как образец блока "Профессиональная деятельность" резюме.
 '''

In [ ]:
user_res_prof = f"Составь часть резюме 'Профессиональная деятельность' из диалога рекрутера и соискателя: \n\n{dialog_prof}. Информацию для резюме бери из этого диалога. От себя информацию не придумывай и не добавляй."

In [ ]:
res_prof = answer_index(system_res_prof, user_res_prof)
with open('res_prof.txt', 'w') as file:
    text_to_write = res_prof
    file.write(text_to_write)

In [ ]:
res_prof

'Профессиональная деятельность:\n2022 - по настоящее время  \nООО "Ромашка"  \nПроект: Создание нейро-консультанта по внутренней документации компании  \nСПЕЦИАЛИСТ ПО ПРОГРАММИРОВАНИЮ  \nЗадачи:  \n1) Сбор базы знаний  \n2) Создание алгоритма нейро-консультанта  \n3) Создание телеграм бота  \nДостижения:  \n1) Повышение качества клиентского обслуживания  \n\nООО "Интел-Инвест"  \nПроект: Создание нейронной сети для классификации изображений со стройки  \nПРОГРАММИСТ-РАЗРАБОТЧИК  \nЗадачи:  \n1) Подготовка датасета  \n2) Проведение экспериментов по проработке оптимальной архитектуры нейронной сети  \nДостижения:  \n1) Автоматизация контроля ситуации на строительной площадке'

## Навыки

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system_skills = '''
Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
В настоящий момент тебе необходимы данные для составления блока резюме "Навыки".
Тебе будут писать клиенты. Твоя задача провести разговор с клиентом так, чтобы он сообщил все данные, необходимые для составления
блока резюме "Навыки" обязательно укажи в ответе примеры IT навыков: Python, C++, Java, NumPy, Pandas, Matplotlib, Seaborn,
Librosa, Gensim, Pymorphy2, Scikit-learn, SciPy, Keras, Pytorch, Langchain, OpenAI.
'''

# Для подачи в user
topic0_skills = '''
Вот список вопросов, которые надо задать клиенту. Задавай вопросы по порядку, по одному.
1. Какие у вас ключевые профессиональные навыки?
2. Какие у вас есть технические навыки, связанные с IT?
'''

In [ ]:
dialog_skills = recruiter_conversation(system_skills, topic0_skills)

Клиент: привет

Рекрутер:  Привет! Рад помочь тебе с составлением резюме. Давай начнем с первого вопроса: какие у вас ключевые профессиональные навыки?
Клиент: Python, openAI, LangChain

Рекрутер:  Отлично, спасибо за информацию! А какие у вас есть технические навыки, связанные с IT?
Клиент: Нарример, какие?

Рекрутер:  Отлично, спасибо за предыдущие ответы. Когда я говорю о технических навыках, я имею в виду знание конкретных языков программирования, фреймворков, библиотек и инструментов, которыми вы владеете. Например, вы упомянули Python, openAI и LangChain. Есть ли у вас опыт работы с другими языками программирования или инструментами, такими как C++, Java, NumPy, Pandas, Matplotlib, Seaborn, Librosa, Gensim, Pymorphy2, Scikit-learn, SciPy, Keras, Pytorch и другими?
Клиент: да, владею также NumPy, Pandas, Matplotlib, Seaborn,, Keras,

Рекрутер:  Отлично, спасибо за информацию! Это очень важные навыки. Что-то еще из технических навыков, которые вы хотели бы добавить в свое резюме?
К

In [ ]:
dialog_skills

'привет Привет! Рад помочь тебе с составлением резюме. Давай начнем с первого вопроса: какие у вас ключевые профессиональные навыки?Python, openAI, LangChain Отлично, спасибо за информацию! А какие у вас есть технические навыки, связанные с IT?Нарример, какие? Отлично, спасибо за предыдущие ответы. Когда я говорю о технических навыках, я имею в виду знание конкретных языков программирования, фреймворков, библиотек и инструментов, которыми вы владеете. Например, вы упомянули Python, openAI и LangChain. Есть ли у вас опыт работы с другими языками программирования или инструментами, такими как C++, Java, NumPy, Pandas, Matplotlib, Seaborn, Librosa, Gensim, Pymorphy2, Scikit-learn, SciPy, Keras, Pytorch и другими?да, владею также NumPy, Pandas, Matplotlib, Seaborn,, Keras, Отлично, спасибо за информацию! Это очень важные навыки. Что-то еще из технических навыков, которые вы хотели бы добавить в свое резюме?нет, это все Отлично, спасибо за предоставленную информацию! Благодаря вашим навыкам

### Создание текстовой части резюме: "Навыки"

In [ ]:
# Инструкция для GPT, которая будет подаваться в system для блока "Навыки"
system_res_skills = '''
Ты профессиональный рекрутер, составляющий резюме для трудоустройства клиента в IT компанию.
Используй представленную информацию от клиента для составления блока "Навыки" резюме.
'''

user_res_skills = f"Составь часть резюме 'Навыки' из диалога рекрутера и соискателя: \n\n{dialog_skills}."

res_skills = answer_index(system_res_skills, user_res_skills)

with open('res_skills.txt', 'w') as file:
    file.write(res_skills)

In [ ]:
res_skills

'**Навыки**\n\n- Python\n- openAI\n- LangChain\n- NumPy\n- Pandas\n- Matplotlib\n- Seaborn\n- Keras'

## Объединение частей резюме. Проверка заполнения.

In [ ]:
# Пути к вашим файлам
file_paths = ['/content/res_me.txt', '/content/res_skills.txt', '/content/res_edu.txt', '/content/res_prof.txt']

# Переменная для хранения объединенного содержимого
combined_CV = ''

# Чтение каждого файла и добавление его содержимого в переменную
for file_path in file_paths:
    with open(file_path, 'r') as file:
        combined_CV += file.read() + '\n'  # Добавляем перевод строки между содержимым файлов

In [ ]:
combined_CV

'О СЕБЕ\n\nМеня зовут Иван Иванов.\nЯ - Junior программист. У меня есть опыт работы в области искусственного интеллекта с использованием языка программирования Python.\nВ настоящее время я совершенствую свои навыки в области искусственного интеллекта и программирования на Python.\nВ свободное время мне нравится читать книги и заниматься плаванием.\nМой контактный номер: +79655666666.\nЯ проживаю в городе Москва.\nМой адрес электронной почты: bbb@mail.ru.\n**Навыки**\n\n- Python\n- openAI\n- LangChain\n- NumPy\n- Pandas\n- Matplotlib\n- Seaborn\n- Keras\nОбразование:\nГод окончания: 2020  \nУчебное заведение: Университет искусственного интеллекта\nСпециальности:\n1) Data Science\n\nдополнительные курсы или сертификаты: Курс Python для анализа данных\nПрофессиональная деятельность:\n2022 - по настоящее время  \nООО "Ромашка"  \nПроект: Создание нейро-консультанта по внутренней документации компании  \nСПЕЦИАЛИСТ ПО ПРОГРАММИРОВАНИЮ  \nЗадачи:  \n1) Сбор базы знаний  \n2) Создание алгорит

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# ПРОМПТ в system
system = load_document_text('https://docs.google.com/document/d/12fSAEOmhbkLTHrDq84OCI8Myo0HnHQ575yh02lzCNI8/edit?usp=sharing')
system

'\ufeffТы профессиональный рекрутер по IT специальностям. У тебя большой опыт по составлению грамотных резюме, чтобы они привлекали внимание работодателей в IT сфере.\r\nУ тебя есть образец правильно составленного резюме.\r\n\r\n\r\nК тебе будут поступать резюме выпускников IT университета. \r\nПроверь заполнение каждого блока резюме. Проверь релевантность указанной информации, насколько указанная информация соответствует IT специальности.\r\n\r\n\r\nСравни резюме с образцом правильно составленного резюме.\r\nВ резюме должны быть заполнены блоки, как в образце правильно составленного резюме: Контакты, О себе, Навыки, Профессиональная деятельность, Образование.\r\n\r\n\r\nВ блоке “Навыки” должны быть указаны IT навыки.\r\nЕсли блок “Навыки” не заполнен, обязательно укажи в ответе примеры IT навыков: Python, C++, Java, NumPy, Pandas, Matplotlib, Seaborn, Librosa, Gensim, Pymorphy2, Scikit-learn, SciPy, Keras, Pytorch\r\n\r\n\r\nУкажи, если в каком-нибудь блоке резюме не указана информаци

In [ ]:
# Образец заполненного резюме
resume_complet_example = load_document_text(
    'https://docs.google.com/document/d/16JAVCymtb0Z7f18qNYksn8sV0bBMc0BQ/edit?usp=sharing&ouid=117120344852794586001&rtpof=true&sd=true')
resume_complet_example

'\ufeffИванов Иван\r\nData Scientist\r\nКОНТАКТЫ  \r\n  \r\n\r\n+7(499)-000-00-00                @exampleexampleexample  \r\nexampleexampleexample@mail.ru  \r\nhttps://github.com/example/exampleexample\r\n\r\n\r\nО СЕБЕ\r\n\r\n\r\nЯ Junior специалист в сфере искусственного интеллекта. Имею опыт работы с анализом данных, разработкой нейронных сетей. \r\nАктивно изучаю и работаю с ChatGPT. \r\nПримеры работ представлены на GitHub https://github.com/example/exampleexample\r\n\r\n\r\nНАВЫКИ\r\n\r\n\r\nPython, GPT (языковое моделирование), NumPy, Pandas, Matplotlib, Seaborn, Keras,Tensorflow \r\n\r\n\r\nПРОФЕССИОНАЛЬНАЯ ДЕЯТЕЛЬНОСТЬ\r\n\r\n\r\n01.2023-н.в.\r\nООО «Терра ЭйАй»\r\nПроект: Создание нейро-консультанта по внутренней документации компании\r\n\r\n\r\nСТАЖЕР\r\n\r\n\r\nЗадачи:\r\n* сбор базы знаний\r\n* создание диалога с помощью ChatGPT для ответов на вопросы сотрудников\r\n\r\n\r\nДостижения:\r\n* разработал код для работы алгоритма нейро-консультанта\r\n* разработал код интеграц

In [ ]:
syst = f"{system} \n\nОбразец правильно составленного резюме: {resume_complet_example}"
user = f"Кратко укажи замечания по заполнению этого резюме: {combined_CV}"
resume_feedback = answer_index(syst, user)

In [ ]:
# выводим  рекомендации для финальной доработки резюме
resume_feedback

'1. В блоке "О себе" не указаны навыки и опыт работы, связанные с IT сферой. Также стоит добавить ссылку на профиль в GitHub или другие профессиональные ресурсы.\n\n2. В блоке "Навыки" указаны некоторые IT навыки, но не хватает конкретики и связи с областью искусственного интеллекта. Например, не хватает упоминания о навыках работы с нейронными сетями, а также библиотеками и фреймворками, связанными с искусственным интеллектом.\n\n3. В блоке "Образование" не указаны специализации или курсы, связанные с искусственным интеллектом. Также стоит добавить информацию о проектах или курсах, связанных с программированием и искусственным интеллектом.\n\n4. В блоке "Профессиональная деятельность" указаны проекты, но не хватает деталей о том, какие конкретно задачи были выполнены, какие технологии использовались и какие результаты были достигнуты. Также стоит добавить информацию о проектах, связанных с искусственным интеллектом и программированием.'

# Нейро-HR: Создание сопроводительного письма к вакансии по резюме.

У нас имеется папка на гугл драйв с текстовыми файлами с рекомендациями от HR по составлению сопроводительных писем. Возьмем несколько из них, при помощи gpt обработаем их, получив объединенный список рекомендаций без повторений. Используем эти рекомендации для написания сопроводительного письма по тексту вакансии и резюме.

Ссылка на папку: https://drive.google.com/drive/folders/1DeDUAiJmLwWsQHjS0DFcBzfC0oPhPhRH

In [ ]:
auth.authenticate_user()                                                        # Авторизации на доступ к Google Диску
drive_service = build('drive', 'v3')                                            # Создание объект service для доступа к Google Drive API

In [ ]:
folder_id = '1DeDUAiJmLwWsQHjS0DFcBzfC0oPhPhRH'                                 # Идентификатор папки, в которой находятся файлы
results = drive_service.files().list(q=f"'{folder_id}' in parents").execute()   # Список файлов из указанной папки на Google Диске
files = results.get('files', [])
files = [file for file in files if file['name'].endswith('.docx')]              # Удаляем файлы не имеющие расширение 'docx'

In [ ]:
good_instructions = ['2 сопроводительное письмо.docx',                          # Список инструкций которые мы будем использовать
                     '4 сопроводительное письмо.docx',
                     '7 сопроводительное письмо.docx',
                     '8 сопроводительное письмо.docx']
data = files
files = [item for item in data if item['name'] in good_instructions]

In [ ]:
# вот, что содержит список files
files

[{'kind': 'drive#file',
  'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'id': '1Dlb2z5HWieqk4F1-DSyR0803Tt3-I8Tl',
  'name': '8 сопроводительное письмо.docx'},
 {'kind': 'drive#file',
  'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'id': '1Q-YiBo9taElybuByliolzBtaUo-HzwfV',
  'name': '7 сопроводительное письмо.docx'},
 {'kind': 'drive#file',
  'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'id': '1LLJXhGc4xyrNbVPPddS1dyWlQaFr0kLj',
  'name': '4 сопроводительное письмо.docx'},
 {'kind': 'drive#file',
  'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'id': '1cmoDNKDCD-bPKrq2P22Sgbytocseaqmf',
  'name': '2 сопроводительное письмо.docx'}]

In [ ]:
# создаем список, содержащий только тексты рекомендаций по написанию сопроводительного письма
def extract_texts(files, drive_service):
    # Создаем пустой список для хранения текстов из файлов
    texts = []

    # Проходим по каждому файлу в предоставленном списке
    for file in files:
        try:
            # Создаем запрос на загрузку файла с Google Drive
            request = drive_service.files().get_media(fileId=file['id'])

            # Используем BytesIO для чтения данных файла в память
            file_io = io.BytesIO()

            # Создаем объект для загрузки файла
            downloader = MediaIoBaseDownload(file_io, request)

            # Загружаем файл частями
            done = False
            while not done:
                # Загружаем следующий кусок файла
                _, done = downloader.next_chunk()

            # Перемещаем указатель в начало файла
            file_io.seek(0)

            # Читаем файл с помощью библиотеки python-docx
            document = Document(file_io)

            # Собираем текст из всех параграфов документа
            text = "\n".join(paragraph.text for paragraph in document.paragraphs)

            # Добавляем полученный текст в список texts
            texts.append(text)
        except Exception as e:
            # В случае ошибки выводим сообщение и продолжаем с следующим файлом
            print(f"Ошибка при обработке файла {file['name']}: {e}")

    # Возвращаем список текстов
    return texts

# Использование
extracted_texts = extract_texts(files, drive_service)

Этот код загружает тексты из файлов docx, находящихся на Google Drive, и сохраняет их в список. Каждый элемент списка - это текст одного документа. Обработка ошибок обеспечивает, что даже при возникновении проблемы с одним из файлов, остальные файлы будут обработаны.

In [ ]:
extracted_texts[0]

'Как правильно составить сопроводительное письмо программисту\nДавайте разберемся, зачем нужно сопроводительное письмо и как его написать так, чтобы вас захотели взять на работу.\nНесмотря на то что сопроводительное письмо или cover letter составляется в свободной форме, мало кто действительно умеет писать его правильно и убедительно. Особенно это относится к джунам и чуть меньше — к мидлам.\nЧто такое сопроводительное письмо\nСопроводительное письмо — это небольшой текст, который идет дополнением к резюме. В нем вы должны в двух словах объяснить эйчару, зачем и почему вас брать на работу.\nОтличительные особенности:\nОбъем: 500 — 1000 знаков\nСтиль: официально-деловой*\nО чем: какую пользу вы принесете компании\n*чаще всего, но не всегда\nНемного статистики\nСогласно исследованию ResumeLab, около 83% работодателей утверждают, что они готовы рассмотреть кандидата и даже пригласить его на собеседование, если он написал отличное сопроводительное письмо, но у него недостаточно объемное ре

Подсчитаем, сколько токенов содержат документы, чтобы понимать, какую модель мы должны использовать

In [ ]:
# Создаем экземпляр токенизатора
encoding = tiktoken.get_encoding("cl100k_base")

# Проходим по каждому элементу списка extracted_texts
for index, text in enumerate(extracted_texts):
    token_count = len(encoding.encode(text))  # Токенизируем текст и считаем количество токенов

    # Выводим информацию о количестве токенов в каждом документе
    print(f"Документ {index + 1}: {token_count} токенов")


Документ 1: 3117 токенов
Документ 2: 2091 токенов
Документ 3: 1637 токенов
Документ 4: 3097 токенов


In [ ]:
extracted_texts[0]

'Как правильно составить сопроводительное письмо программисту\nДавайте разберемся, зачем нужно сопроводительное письмо и как его написать так, чтобы вас захотели взять на работу.\nНесмотря на то что сопроводительное письмо или cover letter составляется в свободной форме, мало кто действительно умеет писать его правильно и убедительно. Особенно это относится к джунам и чуть меньше — к мидлам.\nЧто такое сопроводительное письмо\nСопроводительное письмо — это небольшой текст, который идет дополнением к резюме. В нем вы должны в двух словах объяснить эйчару, зачем и почему вас брать на работу.\nОтличительные особенности:\nОбъем: 500 — 1000 знаков\nСтиль: официально-деловой*\nО чем: какую пользу вы принесете компании\n*чаще всего, но не всегда\nНемного статистики\nСогласно исследованию ResumeLab, около 83% работодателей утверждают, что они готовы рассмотреть кандидата и даже пригласить его на собеседование, если он написал отличное сопроводительное письмо, но у него недостаточно объемное ре

## Обработка всех полученных текстов при помощи gpt, создание кратких и понятных инструкций

In [ ]:
system_generalize = '''
Ты - копирайтер - эксперт и HR-специалист. Ты отлично структуририруешь текст и выделяешь ключевые моменты.
Возьми на себя роль редактора, которому предоставили тексты нескольких рекомендаций по написанию сопроводительных писем и на их основе сделай обобщенную инструкцию
по написанию сопроводительного письма.

Информацию бери только из текстов, не придумывай ничего от себя
Ты должен писать в единственном числе повелительного наклонения.
'''

In [ ]:
# Соединяем все тексты в одно сообщение, добавляя к каждому тексту его номер
combined_text = "\n".join([f'Текст {index + 1}: {text}' for index, text in enumerate(extracted_texts)])
general_instruction = answer_index(system_generalize, combined_text, temp=0)

In [ ]:
general_instruction

'Инструкция по написанию сопроводительного письма для IT-специалиста\n\n1. Приветствие\n   - Начните с приветствия, используя форму приветствия, соответствующую стилю вакансии.\n   - Избегайте нестандартных приветствий, таких как "Уважаемый(ая)".\n\n2. Интерес к вакансии\n   - Подчеркните свой интерес к вакансии и компании.\n   - Укажите, почему вы хотите работать именно в этой компании.\n\n3. Профессиональные навыки и опыт\n   - Кратко опишите ваши профессиональные навыки и опыт, соответствующие требованиям вакансии.\n   - Укажите проекты, в которых вы участвовали, и достижения на предыдущих местах работы.\n\n4. Мотивация и энтузиазм\n   - Подчеркните свою мотивацию и энтузиазм работать в данной компании.\n   - Укажите, что вам нравится в культуре компании, проектах и направлении деятельности.\n\n5. Грамотность и стиль\n   - Проверьте письмо на грамматические и орфографические ошибки.\n   - Соблюдайте деловой стиль общения, но при необходимости адаптируйте его под стиль вакансии.\n\n6

## Написание сопроводительного письма по тексту резюме и вакансии

In [ ]:
# берем фейковое резюме из документа:
file_url = 'https://docs.google.com/document/d/1kszDO4qR_7vM-dBjVXjs5dSnqLJn-m-3rPXTL0YUrFE/export?format=txt'
response = requests.get(file_url)
resume = response.text

In [ ]:
resume

'\ufeff**РЕЗЮМЕ**\r\n\r\n\r\n\r\n\r\n**Персональная информация**\r\nИмя и Фамилия: Максим Иванов\r\nАдрес: г. Москва, ул. Ленина, д. 999, кв. 999 \r\nКонтакты: +7(499)123-45-67, maximivanov@example.com\r\nLinkedIn: linkedin.com/in/example\r\n\r\n\r\n\r\n\r\n**Цель**\r\nМотивированный Junior DS-инженер с годовым опытом в анализе данных, разработке моделей предсказаний и машинном обучении. Нацелен на углубление навыков и развитие в области Data Science.\r\n\r\n\r\n**Опыт работы**\r\n\r\n\r\n*Junior Data Scientist – XYZ Corp, Москва, Июнь 2020 – Настоящий момент*\r\n- Проектирование и реализация моделей предсказательного анализа\r\n- Обработка и анализ больших данных для выявления тенденций и их визуализация с использованием Python, R, SQL и Tableau\r\n- Строительство и отладка алгоритмов машинного обучения и AI\r\n\r\n\r\n**Образование**\r\n\r\n\r\n*Тверской Государственный Университет – Бакалавр компьютерных наук, 2016 – 2020*\r\n\r\n\r\n**Навыки**\r\n- Python, R и SQL\r\n- Навыки работ

Список с сылками на файлы с текстами вакансий полученных с `HH.ru` и `telegram`-каналов

In [ ]:
folder_id = '1h5v3HDBX-A9KaL2QMbpbgdY9rOB2B2rD'                                 # Идентификатор папки, в которой находятся файлы
results = drive_service.files().list(q=f"'{folder_id}' in parents").execute()   # Список файлов из указанной папки на Google Диске
vacancies = results.get('files', [])
vacancies = [vacanc for vacanc in vacancies if vacanc['name'].endswith('.txt')] # Удаляем файлы не имеющие расширение 'txt'

In [ ]:
vacancies

[{'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1fHhz9mwUk80msizLUH87A64knKEXWHbG',
  'name': 'Вакансия01.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1YaHhrWfkU5e-zZTCSYUHGecbPngP15tM',
  'name': 'Вакансия05.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1D6xq4BLafRWcGlejTtshGeN8b8eOubsy',
  'name': 'Вакансия06.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1pq7gHI8oxc5V1mNkoecZ_i6C1Nnq1Otl',
  'name': 'Вакансия03.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '14O5mOapCGyhoTd31eb1WJRrvHWYzO2lc',
  'name': 'Вакансия04.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '1RC0jRoRMKdEtm43Ix_65k2x2UWC70r-z',
  'name': 'Вакансия07.txt'},
 {'kind': 'drive#file',
  'mimeType': 'text/plain',
  'id': '10saGxyc6q_LHlbB3qOSqsqbi69EBkBew',
  'name': 'Вакансия02.txt'}]

In [ ]:
def extract_texts_vacancies(files, drive_service):
    # Создаем пустой список для хранения текстов из файлов
    texts = []

    # Проходим по каждому файлу в предоставленном списке
    for file in files:
        try:
            # Создаем запрос на загрузку файла с Google Drive
            # Здесь fileId - это уникальный идентификатор файла на Google Drive
            request = drive_service.files().get_media(fileId=file['id'])

            # Используем BytesIO для временного хранения данных файла в памяти
            file_io = io.BytesIO()

            # Создаем объект для загрузки файла, который будет скачивать файл по частям
            downloader = MediaIoBaseDownload(file_io, request)

            # Загружаем файл частями. Это может быть необходимо для больших файлов.
            done = False
            while not done:
                # Загружаем следующий кусок файла
                _, done = downloader.next_chunk()

            # После завершения загрузки перемещаем указатель в начало файла
            file_io.seek(0)

            # Чтение содержимого текстового файла. Предполагается, что файл в формате UTF-8.
            # Если формат отличается, необходимо указать правильную кодировку в decode.
            text = file_io.getvalue().decode('utf-8')

            # Добавляем полученный текст в список texts
            texts.append(text)
        except Exception as e:
            # В случае ошибки выводим сообщение об ошибке и продолжаем с следующим файлом
            print(f"Ошибка при обработке файла {file['name']}: {e}")

    # Возвращаем список с текстами всех обработанных файлов
    return texts
extracted_vacancies = extract_texts_vacancies(vacancies, drive_service)

In [ ]:
# в extracted_vacancies содержится список из 7 вакансий
extracted_vacancies[5]

'NLP Engineer\r\nв Replika\r\n5 000 —\u200d 8 000 $/мес на руки\r\n📍 СНГ, Европа, АзияПолная удалёнкаПомощь с переездом\r\nСпециализация\r\nData Science / Machine Learning\r\nУровень\r\nSenior\r\n\r\nReplika — самый популярный в мире чат-бот на английском. Это AI-друг, который хранит воспоминания пользователя и помогает справиться со сложными жизненными ситуациями. Replika получает более 10 млн сообщений от сотен тысяч активных пользователей каждый день. Основная аудитория — США.\r\n\r\nМы помогаем людям чувствовать себя лучше: менее одиноко и более уверенно. Основная ценность нашего продукта — качество диалога. Мы постоянно экспериментируем с тем, как сделать диалог более эмпатичным, проактивным, увлекательным и персонализированным.\r\n\r\nНаша цель — быть лидерами в области Conversational AI, постоянно улучшая ключевые характеристики open-domain диалога.\r\nТехнологии/инструменты\r\nPythonPytorchDockerTransformers\r\nЗачем работать в Replika\r\n\r\n    Мы делаем жизнь миллионов людей

Таким образом, у нас есть резюме, инструкция по написанию сопроводительного письма и тексты 7 вакансий. На основании этого сгенерируем сопроводительные письма для каждой вакансии.

In [ ]:
system_promt = '''
Ты HR-специалист и должен написать сопроводительное письмо к ВАКАНСИИ используя данные из РЕЗЮМЕ.
В сопроводительном письме обязательно обрати внимание на ключевые навыки, языки программирования, библиотеки и методы исследования, совпадающие в ВАКАНСИИ и в РЕЗЮМЕ.
Подчеркни насколько опыт соискателя в РЕЗЮМЕ подходит требованиям ВАКАНСИИ.
Напиши текст не меньше 180 и не больше 220 слов.
При написании письма придерживайся следующей инструкции по написанию сопроводительных писем:

'''
system_united = system_promt + general_instruction

In [ ]:
system_united

'\nТы HR-специалист и должен написать сопроводительное письмо к ВАКАНСИИ используя данные из РЕЗЮМЕ.\nВ сопроводительном письме обязательно обрати внимание на ключевые навыки, языки программирования, библиотеки и методы исследования, совпадающие в ВАКАНСИИ и в РЕЗЮМЕ.\nПодчеркни насколько опыт соискателя в РЕЗЮМЕ подходит требованиям ВАКАНСИИ.\nНапиши текст не меньше 180 и не больше 220 слов.\nПри написании письма придерживайся следующей инструкции по написанию сопроводительных писем:\n\nИнструкция по написанию сопроводительного письма для IT-специалиста\n\n1. Приветствие\n   - Начните с приветствия, используя форму приветствия, соответствующую стилю вакансии.\n   - Избегайте нестандартных приветствий, таких как "Уважаемый(ая)".\n\n2. Интерес к вакансии\n   - Подчеркните свой интерес к вакансии и компании.\n   - Укажите, почему вы хотите работать именно в этой компании.\n\n3. Профессиональные навыки и опыт\n   - Кратко опишите ваши профессиональные навыки и опыт, соответствующие требов

In [ ]:
cover_letters = []
# Проходим по каждому элементу списка
for vacancy in extracted_vacancies:
    # Формируем текст запроса, объединяя текст резюме и описание вакансии
    request_text = 'ТЕКСТ РЕЗЮМЕ: ' + resume + ' ТЕКСТ ВАКАНСИИ: ' + vacancy

    # Используем функцию answer_index для получения сопроводительного письма
    cl = answer_index(system_united, request_text, temp=0)

    # Добавляем сопроводительное письмо в список
    cover_letters.append(cl)

In [ ]:
# вот, какие получились сопроводительные по каждой вакансии:
cover_letters

['Привет,\n\nЯ заинтересован в возможности присоединиться к команде Ventra в качестве Data Scientist. Мой опыт работы в XYZ Corp в области анализа данных и машинного обучения, а также мои навыки в Python, R, SQL, и библиотеках Pandas, NumPy, и Scikit-learn, делают меня подходящим кандидатом для этой роли.\n\nВ XYZ Corp я занимался проектированием и реализацией моделей предсказательного анализа, обработкой больших данных для выявления тенденций, и построением алгоритмов машинного обучения с использованием Python, R, SQL, и Tableau. Мой опыт также включает работу с библиотеками Pandas, NumPy, и Scikit-learn, а также знание статистических методов и моделей. Кроме того, я имею базовые знания Spark и опыт работы с большими данными.\n\nЯ увлечен возможностью работать в Ventra, особенно потому что компания предлагает широкий спектр услуг по аутсорсингу бизнес-процессов и управлению проектами, что позволит мне применить мои навыки в разнообразных областях. Я также заинтересован в возможности р

# Оценка соответствия резюме требованиям вакансии

При помощи инструкции оценим, насколько резюме сосикателя соответствует требованиям вакансии по 5-бальной шкале с пояснением причин.

In [ ]:
system_promt_match = '''
Ты - опытный HR, который очень точно умеет оценивать соответствия РЕЗЮМЕ требованиям ВАКАНСИИ.
Твоя задача - оценить по шкале от 0 до 100 соответствие РЕЗЮМЕ требованиям ВАКАНСИИ по следующим параметрам:

1. Знание языков программирования, фреймворков, библиотек, инструментов и методов.
Отрази насколько знания указанные в РЕЗЮМЕ соответствуют требованиям в ВАКАНСИИ.

2. Навыки и компетенции.
Важно соответствие навыков и компетенции в РЕЗЮМЕ, таких как умение работать в команде,
общие разработческие навыки, аналитическое мышление и способность к самообучению, требованиям указанным в ВАКАНСИИ

3. Проекты и достижения: информация о проектах, выполненных в рамках обучения или личных инициатив
Насколько участие в проектах, соревнованиях, конференциях или других мероприятиях указанных в РЕЗЮМЕ
соответствует направлению работ в ВАКАНСИИ.

4. Опыт работы: РЕЗЮМЕ должно отражать соответствующий опыт работы, который требуется для данной ВАКАНСИИ.
Отрази полноту соответствия опыта работы, выполняемых задач и достижений в РЕЗЮМЕ требуемым обязанностям по работе в ВАКАНСИИ.

5. Образование: Насколько полученное образование в РЕЗЮМЕ подходит под требования ВАКАНСИИ.
По каждому пункту оцени соответствие по шкале от 0 до 100, где 0 - полное несоответствие резюме соискателя требованиям вакансии,
а 100 - это полное соответствие.
ИТОГО: Поставь итоговую оценку и прокомментируй свое решение.
Напиши, где РЕЗЮМЕ соответствует ВАКАНСИИ, а где РЕЗЮМЕ не соответствует ВАКАНСИИ.
'''

In [ ]:
match_results = []
# Проходим по каждому элементу списка
for vacancy in extracted_vacancies:
    # Формируем текст запроса, объединяя текст резюме и описание вакансии
    request_text = 'ТЕКСТ РЕЗЮМЕ: ' + resume + ' ТЕКСТ ВАКАНСИИ: ' + vacancy

    # Используем функцию answer_index для получения сопроводительного письма
    res = answer_index(system_promt_match, request_text, temp=0)

    # Добавляем сопроводительное письмо в список
    match_results.append(res)

In [ ]:
# по каждой вакансии можно посмотреть отчет о соответствии
match_results

['Оценка соответствия резюме требованиям вакансии:\n\n1. Знание языков программирования, фреймворков, библиотек, инструментов и методов.\n   - Python, R, SQL - соответствует\n   - Pandas, NumPy, Scikit-Learn - соответствует\n   - Spark, Pyspark - не указано в резюме\n   - PyTorch, TensorFlow - не указано в резюме\n   - Базовые навыки DevOps (Docker, Kubernetes и др.) - не указано в резюме\n   - Data Factory, Airflow - не указано в резюме\n   - Опыт проработки задачи от бизнес-постановки до математической формулировки и реализации в коде - частично соответствует\n   - Умение презентовать результаты анализа - не указано в резюме\n   - Опыт работы в FMCG или телеком компании - не указано в резюме\nОценка: 60\n\n2. Навыки и компетенции.\n   - Навыки работы с данными, алгоритмы машинного обучения, визуализация данных - соответствует\n   - Внимание к деталям, способность работать над мультидисциплинарными проектами, быстрое обучение и адаптация к новым технологиям, хорошие коммуникативные на

Попробуем запустить то же самое на gpt 4, оценим результаты

In [ ]:
def answer_index_4(system, topic, temp=0):

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": topic}
    ]
    completion = client.chat.completions.create(
        model="gpt-4-0613",
        messages=messages,
        temperature=temp
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

In [ ]:
# при помощи tqdm выведем прогресс выполнения ячейки
from tqdm import tqdm
match_results_gpt4 = []
# Проходим по каждому элементу списка
for vacancy in tqdm(extracted_vacancies):
    # Формируем текст запроса, объединяя текст резюме и описание вакансии
    request_text = 'ТЕКСТ РЕЗЮМЕ: ' + resume + ' ТЕКСТ ВАКАНСИИ: ' + vacancy

    # Используем функцию answer_index для получения сопроводительного письма
    res = answer_index_4(system_promt_match, request_text, temp=0)

    # Добавляем сопроводительное письмо в список
    match_results_gpt4.append(res)

100%|██████████| 7/7 [10:07<00:00, 86.84s/it]


In [ ]:
# результаты использования gpt 4 гораздо точнее соответствуют инструкциям
match_results_gpt4

['1. Знание языков программирования, фреймворков, библиотек, инструментов и методов: 70/100\nВ резюме указаны Python, R, SQL, Pandas, NumPy, Scikit-learn, Hadoop, Spark, что в целом соответствует требованиям вакансии. Однако, в вакансии также указаны PyTorch, TensorFlow, Data Factory, Airflow, Docker, Kubernetes, которые в резюме не упоминаются.\n\n2. Навыки и компетенции: 80/100\nВ резюме указаны навыки анализа данных, проблемного решения, внимание к деталям, способность работать над мультидисциплинарными проектами, быстрое обучение и адаптация к новым технологиям, хорошие коммуникативные навыки. Это в целом соответствует требованиям вакансии, однако, в вакансии также указаны навыки автоматизации пайплайнов данных и базовые навыки DevOps, которые в резюме не упоминаются.\n\n3. Проекты и достижения: 50/100\nВ резюме указан опыт работы в компании XYZ Corp, где выполнялись задачи по проектированию и реализации моделей предсказательного анализа, обработке и анализу больших данных, строите